<a href="https://colab.research.google.com/github/fedefliguer/bot_twitter_UnaTapaComoHoy/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from datetime import date
import requests
from bs4 import BeautifulSoup
import re
import urllib.request
import random
import os
import tweepy as tw
import tempfile
from werkzeug.utils import secure_filename
import numpy as np
from math import log

def tweet(self):
    today = date.today()
    newDay = "{0:0=2d}".format(today.day)
    newMonth = "{0:0=2d}".format(today.month)
    array = np.random.choice(3, 1, p=[0.35, 0.35, 0.3])
    diario = array[0]
    if(diario==0):
        allLags = np.arange(4,74)
        probs = [1/log(y,10) for y in allLags]
        probs = np.square(probs)
        probs = probs/sum(probs)
        lags = np.random.choice(allLags, 1, p=probs)[0] - 1
        newYear = str(today.year - lags)
        url = "http://tapas.clarin.com/tapa/" + newYear + "/" + newMonth + "/" + newDay + "/" + newYear + newMonth + newDay + "_thumb.jpg"

    elif(diario==1):
        allLags = np.arange(5,19)
        probs = [1/log(y,10) for y in allLags]
        probs = np.square(probs)
        probs = probs/sum(probs)
        lags = np.random.choice(allLags, 1, p=probs)[0] - 1
        newYear = today.year - lags
        if newYear == 2016:
          newYear = "2015"
          lags = 5
        if lags < 4:
          newYear = str(newYear)  
          url = "https://www.pagina12.com.ar/edicion-impresa/" + newDay + "-" + newMonth + "-" + newYear
          req = requests.get(url, allow_redirects=True)
          soup = BeautifulSoup(req.text,  "html.parser")
          url = soup.findAll('img',{"class":"lazyload "})
          if len(url)>0:
            url = url[0]["data-src"]
          else:
            diario = 2  
        else:
          newYear = str(newYear)  
          url = "https://www.pagina12.com.ar/diario/principal/diario/index-" + newYear + "-" + newMonth + "-" + newDay + ".html"
          req = requests.get(url, allow_redirects=True)
          soup = BeautifulSoup(req.text,  "html.parser")
          alt = "Tapa de la fecha " + newDay + "-" + newMonth + "-" + newYear
          url = soup.findAll('img',{"alt":alt})
          if len(url)>0:
            url = url[0]["src"]
          else:
            diario = 2
            
    elif(diario==2):
        allLags = np.arange(5,14)
        probs = [1/log(y,10) for y in allLags]
        probs = np.square(probs)
        probs = probs/sum(probs)
        lags = np.random.choice(allLags, 1, p=probs)[0] - 1
        newYear = str(today.year - lags)
        url = "https://tapas2.ole.com.ar/tapa/" + newYear + "/" + newMonth + "/" + newDay + "/" + "OLE_" + newYear + newMonth + newDay + "_03.jpg"

    completeURL = url
    consumer_key = ''
    consumer_secret = ''
    access_token = ''
    access_token_secret = ''

    auth = tw.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tw.API(auth)
    filename = 'temp.jpg'
    file_name = secure_filename(filename)
    path_name = os.path.join(tempfile.gettempdir(), file_name)
    message = newYear
    url = completeURL
    request = requests.get(url, stream=True)
    if request.status_code == 200:
        with open(path_name, 'wb') as image:
            for chunk in request:
                image.write(chunk)

        api.update_with_media(path_name, status=message)
        os.remove(path_name)
    else:
        print("Unable to download image")
    return f'Yes!'